## Deep learning approach

In [42]:
#Deep learning library of choice PyTorch

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split

# for number-crunching

import numpy as np
import scipy.stats as stats
import pandas as pd

# Time to check that the gpu optimization is actually helping

import time 

# Some graphing

import matplotlib.pyplot as plt

In [43]:
# use GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [44]:
df = pd.read_csv('../datelets/datalet2.csv',index_col=False)
df = df.astype('int')


In [45]:
display(df)

,period0,period1
0,3,2
1,2,0
2,0,2
3,2,1
4,1,1
...,...,...
9999993,3,3
9999994,3,2
9999995,2,0
9999996,0,3


In [46]:
df['period0'] = df['period0'].apply(lambda x: 'high' if x == 3 else x )
df['period0'] = df['period0'].apply(lambda x: 'med' if x == 2 else x)
df['period0'] = df['period0'].apply(lambda x: 'low' if x == 1 else x)
df['period0'] = df['period0'].apply(lambda x: 'none' if x == 0  else x)

In [47]:
dummies = pd.get_dummies(df.period0)

df = pd.concat([df, dummies], axis='columns')

df.drop(['period0'], axis = 'columns')

,period1,high,low,med,none
0,2,1,0,0,0
1,0,0,0,1,0
2,2,0,0,0,1
3,1,0,0,1,0
4,1,0,1,0,0
...,...,...,...,...,...
9999993,3,1,0,0,0
9999994,2,1,0,0,0
9999995,0,0,0,1,0
9999996,3,0,0,0,1


In [48]:
dataheaders = dummies.columns

data = torch.Tensor(df[dataheaders].values).type(torch.float)
labels = torch.Tensor(df['period1'].values).type(torch.LongTensor)

In [49]:
print(data)
print(labels)

tensor([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        ...,
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]])
tensor([2, 0, 2,  ..., 0, 3, 2])


In [50]:
# First we are going to split the data into three parts, evaluation, test and training data. I do this with two instances of train_test_split, just for the sake of convinience.

traintemp_data,eval_data, traintemp_labels,eval_labels = train_test_split(data, labels, test_size=.01)

train_data,test_data, train_labels,test_labels = train_test_split(traintemp_data, traintemp_labels, test_size=.01)

#then we are going to pass the data to the Pytorch data loader, this is going to allow us to split it into mini batches that will be run through the model.
#given that we are working with 10mil data points this is essential or we would simply run out of memory on the devices. Im using 2048 in the hope there are some gains to be made with that matrix size.

train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

train_data = train_data
test_data = test_data

batchsize    = 2084*2*2*2*2
train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True, num_workers=12)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0]) 

In [51]:
# create a class for the model WITH BATCH NORM

class ANN(nn.Module):
	def __init__(self, Input_dim, Output_dim):
		super().__init__()

		### input layer
		self.input = nn.Linear(Input_dim,4)
		
		### hidden layers
		self.hidden1    = nn.Linear(4,4)
		self.bnorm1 = nn.BatchNorm1d(4) 
		self.hidden2    = nn.Linear(4,4)
		self.bnorm2 = nn.BatchNorm1d(4) 
		self.hidden3    = nn.Linear(4,4)
		self.bnorm3 = nn.BatchNorm1d(4)
		self.hidden4    = nn.Linear(4,4)

		### output layer
		self.output = nn.Linear(4,Output_dim)
	
	# forward pass
	def forward(self,x):

		# input (x starts off normalized)
		x = F.relu( self.input(x) )


		# hidden layer 1
		x = self.bnorm1(x) # batchnorm
		x = F.relu( self.hidden1(x) )      # linear function and activation function

		# hidden layer 2
		x = self.bnorm2(x) # batchnorm
		x = F.relu( self.hidden2(x) )      # linear function and activation function
		
		# hidden layer 3
		x = self.bnorm3(x)
		x = F.relu( self.hidden3(x) )      # linear function and activation function
  
		# hidden layer 4

		x = F.relu( self.hidden4(x) )

		# output layer
		return self.output(x)

In [52]:
def trainthemodel():
	
	# Loss function and optimizer, I chose the BCE loss function as it is best for classification problems. 
	lossfun = nn.CrossEntropyLoss()
	optimizer = torch.optim.SGD(model.parameters(),lr=0.01)
	
	#initialize losses
	losses = torch.zeros(numofepochs)
	trainAcc = []
	testAcc = []

	model.to(device)
	
	#now lets actually loop over the training epochs to train the model
	for epoch in range(numofepochs):
		
		# switch on training mode
		model.train()

		# loop over training data batches
		batchAcc  = []
		batchLoss = []
		for X,y in train_loader:

			X = X.to(device)
			y = y.to(device)
			
			# forward pass and loss
			yHat = model(X)
			loss = lossfun(yHat,y)

			# backprop
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()

			# loss from this batch
			batchLoss.append(loss.item())

			yHat = yHat.cpu()
			y = y.cpu()

			# compute training accuracy for this batch
			batchAcc.append( 100*torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
			
		# now that we've trained through the batches, get their average training accuracy
		trainAcc.append( np.mean(batchAcc)) 

		# and get average losses across the batches
		losses[epoch] = np.mean(batchLoss)
		
		### test accuracy

		# Lets turn eval back on so we dont overfit with the test data 
		model.eval()
		X,y = next(iter(test_loader)) # extract X,y from test dataloader

		X = X.to(device)
		y = y.to(device)  

		with torch.no_grad(): # deactivates autograd
			yHat = model(X)
   
		yHat = yHat.cpu()
		y = y.cpu()   

		testAcc.append( 100*torch.mean((torch.argmax(yHat,axis=1) == y).float()) )

		print('epoch {} done at time {} '.format(epoch,time.perf_counter()))


	# function output
	return trainAcc,testAcc,losses,model

In [53]:
input_dim = dummies.shape[1]
output_dim = 4
numofepochs = 1000


model = ANN(Input_dim = input_dim,Output_dim = output_dim)
trainAcc,testAcc,losses,model = trainthemodel()

epoch 0 done at time 892.4802683 
epoch 1 done at time 908.800765 
epoch 2 done at time 924.5166726 
epoch 3 done at time 940.8974184 
epoch 4 done at time 956.8222369 
epoch 5 done at time 973.0317618 
epoch 6 done at time 989.0345298 
epoch 7 done at time 1004.7260482 
epoch 8 done at time 1020.5900878 
epoch 9 done at time 1036.3892069 
epoch 10 done at time 1052.241094 
epoch 11 done at time 1068.1593944 
epoch 12 done at time 1084.0736771 
epoch 13 done at time 1099.854123 
epoch 14 done at time 1115.6518514 
epoch 15 done at time 1132.0190929 
epoch 16 done at time 1148.1780221 
epoch 17 done at time 1164.2599709 
epoch 18 done at time 1180.2345049 
epoch 19 done at time 1196.1643731 


In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,5))



ax[0].plot(losses,'k^-')
ax[0].set_ylabel('Loss')
ax[0].set_xlabel('Epochs')
ax[0].set_title('Losses with minibatch size=' + str(batchsize))

ax[1].plot(trainAcc,)
ax[1].plot(testAcc,)
ax[1].set_title('Accuracy with minibatch size=' + str(batchsize))
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy (%)')
ax[1].legend(['Train','Test'])
ax[1].set_ylim([0,103])

plt.show()

